In [1]:
# Instalar libreria
!pip install folium

In [84]:
import pandas as pd
import folium
from folium.plugins import HeatMap

In [85]:
# Se cargaran 2 datasets. Primero los datos de ventas
sales_data = pd.read_csv('50000_Sales_Records.csv')

In [86]:
# Ahora los datos de coordenadas
coordinates_data = pd.read_csv('world_coordinates.csv')

In [87]:
sales_data.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Namibia,Household,Offline,M,8/31/2015,897751939,10/12/2015,3604,668.27,502.54,2408445.08,1811154.16,597290.92
1,Europe,Iceland,Baby Food,Online,H,11/20/2010,599480426,1/9/2011,8435,255.28,159.42,2153286.80,1344707.70,808579.10
2,Europe,Russia,Meat,Online,L,6/22/2017,538911855,6/25/2017,4848,421.89,364.69,2045322.72,1768017.12,277305.60
3,Europe,Moldova,Meat,Online,L,2/28/2012,459845054,3/20/2012,7225,421.89,364.69,3048155.25,2634885.25,413270.00
4,Europe,Malta,Cereal,Online,M,8/12/2010,626391351,9/13/2010,1975,205.70,117.11,406257.50,231292.25,174965.25


In [88]:
coordinates_data.head()

,Code,Country,Latitude,Longitude
0,AD,Andorra,42.546245,1.601554
1,AE,United Arab Emirates,23.424076,53.847818
2,AF,Afghanistan,33.939110,67.709953
3,AG,Antigua and Barbuda,17.060816,-61.796428
4,AI,Anguilla,18.220554,-63.068615


In [91]:
# Filtrar las columnas necesarias y aplicar filtros 
filtered_sales_data = sales_data[(sales_data['Sales Channel'] == 'Online') & (sales_data['Order Priority'] == 'H') & (sales_data['Region'] == 'Europe')]

In [94]:
# Agrupar por pais y sumar las ventas totales por cada uno
filtered_sales_data = filtered_sales_data[['Country', 'Total Revenue']].groupby('Country').sum().reset_index()

In [95]:
# Renombrar Total Revenue como ventas
filtered_sales_data.rename(columns = {'Total Revenue':'Sales'}, inplace = True)

In [96]:
# Unir el dataset de ventas con el dataset de coordenadas
merged_data = pd.merge(filtered_sales_data, coordinates_data, on = 'Country', how = 'left')

In [97]:
# Eliminar filas con coordenadas faltantes
merged_data.dropna(subset = {'Latitude', 'Longitude'}, inplace = True)

In [98]:
merged_data.columns

Index(['Country', 'Sales', 'Code', 'Latitude', 'Longitude'], dtype='object')

In [99]:
# Crear el mapa basado en Europa
mapa = folium.Map(location = [54.5260,15.2551], zoom_start = 4)

In [100]:
mapa

In [106]:
# Agregar los datos al mapa
for index, row in merged_data.iterrows():
    folium.CircleMarker(location = [row['Latitude'], row['Longitude']],
                       radius = row['Sales']*0.00001, # Ajustar el factor de escala según sea necesario
                       popup = f"Country: {row['Country']}<br>Sales: {row['Sales']}",
                       color = 'blue',
                       fill = True,
                       fill_color = 'blue').add_to(mapa)
mapa

## Crear un Mapa de Calor

In [107]:
# Preparar los datos generando una lista de listas donde cada sublista contenga un valor de peso, latitude y longitud
heat_data = [[row['Latitude'], row['Longitude'], row['Sales'] * 0.00001] for index, row in merged_data.iterrows()]

In [109]:
# Creando el mapa de Calor
mapa_heat = folium.Map(location = [54.5260,15.2551], zoom_start = 4)
HeatMap(heat_data).add_to(mapa_heat)

In [110]:
mapa_heat